In [3]:
import os
import sys
sys.path.append("../")

import pandas as pd
from tqdm import tqdm

from langchain.document_loaders import PyPDFLoader

from utils.arxiv_utils import get_inspire_hep_papers, extract_arxiv_ids, download_arxiv_source, remove_latex_preamble
from utils.db_utils import update_dataframe, delete_files_except_extensions, get_filenames_with_extensions, scrape_website_text

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
pdf_dir = '../data/papers/'  # directory to store PDFs
db_dir = '../data/db/'  # directory to store database CSVs
txt_dir = '../data/interviews/'  # directory to store interview transcripts

## 1. Papers

In [5]:
try:
    os.makedirs(pdf_dir)
except FileExistsError:
    print(f"Directory '{pdf_dir}' already exists")

# Get papers from INSPIRE-HEP, with a year cutoff
papers = get_inspire_hep_papers("Jesse.Thaler.1",year_cutoff=2018) 

# Extract arXiv IDs from papers
arxiv_ids = extract_arxiv_ids(papers) + ["1402.2657", "1011.2268"]

# Download papers (sources if available, otherwise PDFs)
[download_arxiv_source(arxiv_id, output_dir=pdf_dir) for arxiv_id in tqdm(arxiv_ids)];  

Directory '../data/papers/' already exists


  2%|█                                                        | 1/55 [00:10<09:51, 10.95s/it]

Found 1 .tex files in 1908.08542
They are: ['../data/papers/1908.08542/mod_emd.tex']
Using ../data/papers/1908.08542/mod_emd.tex as the main .tex file


  4%|██                                                       | 2/55 [00:13<05:28,  6.20s/it]

Found 1 .tex files in 2004.04159
They are: ['../data/papers/2004.04159/eventgeometry.tex']
Using ../data/papers/2004.04159/eventgeometry.tex as the main .tex file


...

Found 1 .tex files in 1804.03657
They are: ['../data/papers/1804.03657/rsd_v2.tex']
Using ../data/papers/1804.03657/rsd_v2.tex as the main .tex file


 98%|██████████████████████████████████████████████████████▉ | 54/55 [06:19<00:03,  3.49s/it]

Found 1 .tex files in 1402.2657
They are: ['../data/papers/1402.2657/softdrop_paper.tex']
Using ../data/papers/1402.2657/softdrop_paper.tex as the main .tex file


100%|████████████████████████████████████████████████████████| 55/55 [06:20<00:00,  6.92s/it]

Found 1 .tex files in 1011.2268
They are: ['../data/papers/1011.2268/paper.tex']
Using ../data/papers/1011.2268/paper.tex as the main .tex file


In [6]:
# Delete all files except PDFs and TeX files; load relevant files list
delete_files_except_extensions(pdf_dir, ['.pdf', '.tex'])

In [7]:
filenames = get_filenames_with_extensions(pdf_dir, ['.tex', '.pdf'])

In [8]:
# Get text
source_type = []
text = []

for i, file in enumerate(tqdm(filenames)):
    try:
        if os.path.splitext(file)[-1] == '.pdf':
            loader = PyPDFLoader("{}/{}".format(pdf_dir, file))
            pages = loader.load_and_split()
            text.append(''.join([page.page_content for page in pages]))
            source_type.append("paper")
        elif os.path.splitext(file)[-1] == '.tex':
            with open("{}/{}".format(pdf_dir, file), 'r', encoding='iso-8859-1') as f:
                text.append(remove_latex_preamble(f.read()))
                source_type.append("paper")
    except:
        print("Error with file {}".format(file))

100%|████████████████████████████████████████████████████████| 55/55 [00:14<00:00,  3.88it/s]


In [9]:
data = [source_type, text]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [10]:
# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)

## 2. YouTube videos

In [11]:
# from tqdm import tqdm
# import whisper
# import pytube
# from pathlib import Path
# import subprocess
# import numpy as np

In [12]:
# videos_dir = "../data/videos/"

# try:
#     os.makedirs(videos_dir)
# except FileExistsError:
#     print(f"Directory '{videos_dir}' already exists")

In [13]:
# # Get whisper model; download weights if necessary
# whisper_model = whisper.load_model("tiny.en").to('cpu')
# options = whisper.DecodingOptions(language="en", without_timestamps=True)

# url = "https://www.youtube.com/watch?v=dqxdPNzBY0I"
# pytube_vid = pytube.YouTube(url)
# video_path_local = Path(videos_dir).resolve() / (pytube_vid.video_id+".mp4")
# pytube_vid.streams.filter(type="audio", mime_type="audio/mp4", abr="48kbps").first().download(output_path=video_path_local.parent, filename=video_path_local.name)
# video_path_local = video_path_local.with_suffix(".wav")
# result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])
# transcription = whisper.transcribe(whisper_model, str(video_path_local))

## 3. Interviews

In [14]:
filenames = get_filenames_with_extensions(txt_dir, ['.txt'])

In [15]:
source_type = []
text = []

for i, file in enumerate(tqdm(filenames)):
    try:
        with open("{}/{}".format(txt_dir, file), 'r') as f:
            text.append(f.read())
            source_type.append("interview")
    except:
        print("Error with file {}".format(file))

100%|████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 3197.92it/s]


In [16]:
data = [source_type, text]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)

## 4. Website/CV

In [17]:
websites = ["https://jthaler.net/group", "https://jthaler.net/research", "https://jthaler.net/engagement", "https://jthaler.net/faq", "https://jthaler.net/cv", "https://jthaler.net/contact"]
text_website = [scrape_website_text(website) for website in tqdm(websites)]
text_website = [", ".join(filter(None, text.replace("\n", ",").split(","))) for text in text_website]

100%|██████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.22it/s]


In [19]:
data = [len(text_website) * ["website"], text_website]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)

In [20]:
len(df)  # Number of context objects (papers, sites, etc)

68